In [1]:
import numpy
import pandas
import matplotlib.pyplot as plt
import ESN_Modules as ESN
from scipy import linalg
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error

In [2]:
def test_parameters(X, train_dataset, test_dataset, train_length, test_length, transiente, N, K, scale, radius, sparsity, leak):
    numpy.random.seed(X)
    # PESOS DE ENTRADA
    W_input = ESN.input_weights(N = N, K = K, input_scaling = scale)
    # PESOS DE RESERVATORIO
    W_reservoir = ESN.reservoir_weights(N = N, spectral_radius = radius, sparsity = sparsity)
    
    # MATRIZ DE ESTADOS (TRAINNING)
    state_matrix = ESN.harvesting_states(state_matrix = ESN.create_states(dim = 1 + K + N, patterns = train_length - transiente), 
                                         input_patterns = train_dataset, 
                                         win = W_input, 
                                         wres = W_reservoir, 
                                         initial_state = numpy.zeros((N, 1)),
                                         leaking = leak, 
                                         transiente = transiente)

    # PESOS DE SAÍDA
    W_output = ESN.ridge_regression(states = state_matrix, target = train_target)
    
    # O ESTADO INICIAL É O ÚLTIMO ESTADO DO TREINAMENTO
    initial_state = ESN.reshape(state_matrix[K + 1::,-1])

    # MATRIX DE ESTADOS (TESTING)
    state_matrix = ESN.harvesting_states(state_matrix = ESN.create_states(dim = 1 + K + N, patterns = test_length), 
                                         input_patterns = test_dataset, 
                                         win = W_input, 
                                         wres = W_reservoir, 
                                         initial_state = initial_state,
                                         leaking = leak, 
                                         transiente = 0)
    # ERRO DE PREDIÇÃO
    return(numpy.dot(W_output, state_matrix))

In [3]:
# TAMANHO DE JANELA DE ENTRADA (INPUT)
len_window_input = 1
# TAMANHO DE JANELA DE SAÍDA (OUTPUT)
len_window_output = 1

# PASSOS AHEAD QUE DESEJAMOS PREVER
steps = 1

# TAMANHO DA JANELA DE TRAINNING
train_length = 3500
# TAMANHO DA JANELA DE TESTING
test_length = 1500

# TRANSIENTE, VALORES A SEREM DESCARTADOS
transiente = int(train_length*0.1)

# NÚMERO DE NEURÔNIOS NA CAMADA OCULTA, INT, N > 0
neurons = numpy.linspace(start = 20, stop = 500, num = 25).astype(int)

# NÚMERO DE RODADAS
rounds = 10

# ESPARSIDADE DA MATRIZ, FLOAT, NO CONJUNTO [0, 1]
sparsities = [0.75]
# RAIO ESPECTRAL DESEJADO, FLOAT, NO CONJUNTO (0, INF)
spectral_radius = [0.7]
# TAXA DE VAZAMENTO ESPECTRAL, FLOAT, NO CONJUNTO (0, 1]
leaking_rates = [0.95]
# ESCALA DOS DADOS DE ENTRADA, FLOAT, NO CONJUNTO (0, INF)
input_scalings = [0.3]

In [4]:
trainning_data = pandas.read_csv("ParametrosTreino_NoiseMackey22.txt")
trainning_data = trainning_data[(trainning_data.LEAK == leaking_rates[0]) & (trainning_data.SCALE == input_scalings[0]) & (trainning_data.SPARSITY == sparsities[0]) & (trainning_data.RADIUS == spectral_radius[0])]
trainning_data

,ROUND,WINDOW,NEURONS,LEAK,SPARSITY,SCALE,RADIUS,MSE,RMSE,MAPE,MSE(X),RMSE(X),MAPE(X),MSE(Y),RMSE(Y),MAPE(Y),MSE(Z),RMSE(Z),MAPE(Z),FRAMEWORK
9713,0.0,1.0,20.0,0.95,0.75,0.3,0.7,0.000279,0.012822,0.061100,0.000279,0.012822,0.061100,NaN,NaN,NaN,NaN,NaN,NaN,58009.0
19713,1.0,1.0,20.0,0.95,0.75,0.3,0.7,0.000287,0.013086,0.058938,0.000287,0.013086,0.058938,NaN,NaN,NaN,NaN,NaN,NaN,156134.0
29713,2.0,1.0,20.0,0.95,0.75,0.3,0.7,0.000273,0.012762,0.077215,0.000273,0.012762,0.077215,NaN,NaN,NaN,NaN,NaN,NaN,18541.0
39713,3.0,1.0,20.0,0.95,0.75,0.3,0.7,0.000271,0.012888,0.070994,0.000271,0.012888,0.070994,NaN,NaN,NaN,NaN,NaN,NaN,46985.0
49713,4.0,1.0,20.0,0.95,0.75,0.3,0.7,0.000282,0.012889,0.063283,0.000282,0.012889,0.063283,NaN,NaN,NaN,NaN,NaN,NaN,76975.0
59713,5.0,1.0,20.0,0.95,0.75,0.3,0.7,0.000279,0.012787,0.052732,0.000279,0.012787,0.052732,NaN,NaN,NaN,NaN,NaN,NaN,146832.0
69713,6.0,1.0,20.0,0.95,0.75,0.3,0.7,0.000280,0.012911,0.063320,0.000280,0.012911,0.063320,NaN,NaN,NaN,NaN,NaN,NaN,85402.0
79713,7.0,1.0,20.0,0.95,0.75,0.3,0.7,0.000280,0.012815,0.056488,0.000280,0.012815,0.056488,NaN,NaN,NaN,NaN,NaN,NaN,69435.0
89713,8.0,1.0,20.0,0.95,0.75,0.3,0.7,0.000279,0.012803,0.063613,0.000279,0.012803,0.063613,NaN,NaN,NaN,NaN,NaN,NaN,106214.0
99713,9.0,1.0,20.0,0.95,0.75,0.3,0.7,0.000281,0.012928,0.061768,0.000281,0.012928,0.061768,NaN,NaN,NaN,NaN,NaN,NaN,113189.0


In [5]:
# DEFINE UMA SEMENTE DO GERADOR PSEUDO-ALEATORIO
numpy.random.seed(42)

cols = ["ROUND", "WINDOW", "NEURONS", "LEAK", "SPARSITY", "SCALE", "RADIUS", "MSE", "RMSE", "MAPE", 
        "MSE(X)", "RMSE(X)", "MAPE(X)", "MSE(Y)", "RMSE(Y)", "MAPE(Y)", "MSE(Z)", "RMSE(Z)", "MAPE(Z)",
        "FRAMEWORK"]

sample = len(neurons) * rounds

noise_weight = 0.0025

architectures = trainning_data["FRAMEWORK"].astype(int).tolist()
# CORRIGIR DEPOIS
n_architectures = numpy.random.randint(low = 1, high = 10000, size = rounds)
simulation = pandas.DataFrame(numpy.nan, columns = cols, 
                              index = numpy.linspace(start = 0, stop = sample - 1, num = sample).astype(int))
i = 0

#eval_model = train_target[0].T

for rnd in range(rounds):
    
    # INPUT DO SISTEMA
    numpy.random.seed(architectures[rnd])
    input_dataset, K = ESN.set_window(name = "series_mackey22_norm.txt", window_len = len_window_input, begin = 0)

    # OUTPUT ESPERADO
    output_dataset, L = ESN.set_window(name = "series_mackey22_norm.txt", window_len = len_window_output, begin = len_window_input)
    output_dataset += numpy.random.normal(0, noise_weight, output_dataset.shape)

    # DATASET DE TREINO -> [(X0), (X1), (X2), ..., (Xjanela_treino-1)]
    train_dataset = input_dataset[0 : train_length, :]
    # DATASET DE TREINO (TARGET) -> [(Ysteps+transiente), (Ysteps+transiente+1), ..., (Ysteps+janela_treino-1)]
    train_target = output_dataset[None, steps + transiente - 1 : train_length + steps - 1, :]

    # DATASET DE TESTE -> [(Xjanela_treino), (Xjanela_treino+1), (Xjanela_treino+2), ..., (Xjanela_treino+janela_teste-1)]
    test_dataset = input_dataset[train_length : train_length + test_length, :]
    # DATASET DE TESTE (TARGET) -> [(Ysteps+janela_treino+janela_teste), ..., (Ysteps+janela_treino+janela_teste-1)]
    test_target = output_dataset[train_length + steps - 1 : train_length + test_length + steps - 1, :].T
    
    for N in neurons:
        for leak in leaking_rates:
            for sparsity in sparsities:
                for scale in input_scalings:
                    for radius in spectral_radius:
                        predict = test_parameters(
                            X = n_architectures[rnd], 
                            train_dataset = train_dataset, 
                            test_dataset = test_dataset, 
                            train_length = train_length, 
                            test_length = test_length, 
                            transiente = transiente, 
                            N = N, 
                            K = K, 
                            scale = scale, 
                            radius = radius, 
                            sparsity = sparsity, 
                            leak = leak
                        )
                        simulation.iloc[i,:] = [rnd, len_window_input, N, leak, sparsity, scale, radius] + ESN.model_metrics(test_target, predict) + [architectures[rnd]]
                        i += 1

In [6]:
simulation

,ROUND,WINDOW,NEURONS,LEAK,SPARSITY,SCALE,RADIUS,MSE,RMSE,MAPE,MSE(X),RMSE(X),MAPE(X),MSE(Y),RMSE(Y),MAPE(Y),MSE(Z),RMSE(Z),MAPE(Z),FRAMEWORK
0,0.0,1.0,20.0,0.95,0.75,0.3,0.7,0.000077,0.006204,0.031368,0.000077,0.006204,0.031368,NaN,NaN,NaN,NaN,NaN,NaN,58009.0
1,0.0,1.0,40.0,0.95,0.75,0.3,0.7,0.000059,0.005338,0.024000,0.000059,0.005338,0.024000,NaN,NaN,NaN,NaN,NaN,NaN,58009.0
2,0.0,1.0,60.0,0.95,0.75,0.3,0.7,0.000057,0.005311,0.023063,0.000057,0.005311,0.023063,NaN,NaN,NaN,NaN,NaN,NaN,58009.0
3,0.0,1.0,80.0,0.95,0.75,0.3,0.7,0.000052,0.005045,0.021743,0.000052,0.005045,0.021743,NaN,NaN,NaN,NaN,NaN,NaN,58009.0
4,0.0,1.0,100.0,0.95,0.75,0.3,0.7,0.000052,0.005010,0.019230,0.000052,0.005010,0.019230,NaN,NaN,NaN,NaN,NaN,NaN,58009.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,9.0,1.0,420.0,0.95,0.75,0.3,0.7,0.000047,0.004698,0.023550,0.000047,0.004698,0.023550,NaN,NaN,NaN,NaN,NaN,NaN,113189.0
246,9.0,1.0,440.0,0.95,0.75,0.3,0.7,0.000046,0.004649,0.023128,0.000046,0.004649,0.023128,NaN,NaN,NaN,NaN,NaN,NaN,113189.0
247,9.0,1.0,460.0,0.95,0.75,0.3,0.7,0.000047,0.004698,0.023571,0.000047,0.004698,0.023571,NaN,NaN,NaN,NaN,NaN,NaN,113189.0
248,9.0,1.0,480.0,0.95,0.75,0.3,0.7,0.000046,0.004670,0.023181,0.000046,0.004670,0.023181,NaN,NaN,NaN,NaN,NaN,NaN,113189.0


In [7]:
simulation.to_csv("ParametrosTeste_ModeloIIIMackey22.txt", index = False)